### Installing and Importing all the necessary packages

*Hey, We are Siddhant & Pavana and we are participating in Mistral FT Hackathon :3. To begin our Mistral Backend, we need to install and import the necessary libraries. This setup ensures we have all the tools required for data manipulation, visualization, and interaction with the Mistral API with FastAPI backend framework.*

In [1]:
!pip install openai-whisper elevenlabs pyngrok fastapi[all] nest_asyncio mistralai -q
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307

In [2]:
import whisper
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
import locale
from pydantic import BaseModel
from elevenlabs.client import ElevenLabs
import uuid
import os
import asyncio
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from fastapi.responses import FileResponse
import gc
import torch
from elevenlabs import save
from mistralai.models.chat_completion import ChatMessage
from mistralai.client import MistralClient
from google.colab import userdata
locale.getpreferredencoding = lambda: "UTF-8"

client = ElevenLabs(api_key = userdata.get('ELEVENLABS_API_KEY'))
mistralclient = MistralClient(api_key=userdata.get('MISTRAL_API_KEY'))

device = "cuda" if torch.cuda.is_available() else 'cpu'
voice_model = whisper.load_model("large-v3",device=device)
port=8888
gc.collect()

100%|██████████████████████████████████████| 2.88G/2.88G [00:29<00:00, 106MiB/s]


0

### Creating Datamodels and pre-prompts!

*To streamline the process of generating responses, we define specific data models and pre-prompts that structure the input and guide the model's behavior.*

In [3]:
class TTSNER(BaseModel):
    text: str
    emotion: str = "Cheerful & Professional"

In [4]:
preprompt = """You are an intelligent clinical language model.
Below is a snippet of patient's electronic health record note and a following instruction with question from healthcare professional.
Write a response that appropriately completes the instruction.
The response should provide the accurate answer to the instruction, while being concise."""

print(preprompt)

You are an intelligent clinical language model.
Below is a snippet of patient's electronic health record note and a following instruction with question from healthcare professional.
Write a response that appropriately completes the instruction.
The response should provide the accurate answer to the instruction, while being concise.


### Running the Backend and fetching the NGROK Link

*To deploy and access the backend of our Mistral FT Hacks project, we utilize FastAPI to create an API server. This server handles various endpoints for tasks such as transcribing audio, performing text-to-speech (TTS) conversions, and interacting with the fine-tuned Mistral model.*

In [22]:
app = FastAPI(title="Mistral FT Hacks Backend")

app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

@app.get("/")
async def home():
    return "Mistral FT Hack"

@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):#,token: str = Header(...)):
    try:
      #decoded_token = decode_token(token)
      audio_path = f"{uuid.uuid4()}.webm"
      with open(audio_path, "wb") as f:
          f.write(await file.read())
      result = voice_model.transcribe(whisper.pad_or_trim(whisper.load_audio(audio_path)))
      text = result["text"]
      src_lang = result["language"]
      os.remove(audio_path)
      return {"text": text,"src_lang":src_lang}
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/labs-tts/")
async def labs_tts(request: TTSNER = Body(...)):#,token: str = Header(...)):
    try:
      #decoded_token = decode_token(token)
      out = f"{uuid.uuid4()}.ogg"
      async def remove():
          loop = asyncio.get_event_loop()
          await loop.run_in_executor(None, lambda: os.remove(out))
      audio = client.generate(
        text=request.text,
        voice="Paul",
        model="eleven_multilingual_v2"
      )
      save(audio,out)
      return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/mistral/")
async def mistral(request: TTSNER = Body(...)):#,token: str = Header(...)):
    try:
      #decoded_token = decode_token(token)
      result = mistralclient.chat(model='ft:open-mistral-7b:885d8e28:20240630:ddcad168',
                                  messages=[ChatMessage(role='user',
                                  content=preprompt+'\n\n'+request.text)]).dict()["choices"][0]["message"]["content"]
      return {"text":result}
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__=="__main__":
    nest_asyncio.apply()
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    uvicorn.run(app,port=port)

Public URL: https://523e-34-19-19-170.ngrok-free.app
INFO:     Started server process [293]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8888 (Press CTRL+C to quit)
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "OPTIONS /mistral/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "POST /mistral/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "OPTIONS /transcribe/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "POST /transcribe/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "POST /mistral/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:6595:1114:0 - "OPTIONS /labs-tts HTTP/1.1" 200 OK
INFO:     2401:4900:1c97:8f58:b643:dd0f:65